<font size=5  color=#003366> [LINMA2472] - Algorithms in Data Science <br>
Project 1: Networks </font> <br><br>
<font size=3  color=#003366>  Students: Iacopo Canetta, Giovanni Faldani, Constance Nkweya Tofeun
<br><br>

- <font size=4  color=#003366>Task 1: building the co-occurence network of characters </font> <br><br>
In this project we'll build a graph network representing the bonds between characters present in the classical novel "The Betrothed" by Alessandro Manzoni

In [ ]:
# basic imports
import numpy as np
from numpy import linalg as LA
import pandas as pd # useful dataframe support
import scipy as sp
from matplotlib import pyplot as plt # plot operations
import networkx as nx # useful for network operations
import nltk # useful for sentence parsing
import re # regular expressions for text parsing and finding proper names
import networkx.algorithms.community as nx_comm
import sklearn
from sklearn.cluster import SpectralClustering
import communities.algorithms as comm
nltk.download('punkt')

In [ ]:
# text file parsing
#txt_filename = "./data/pg35155.txt"
txt_filename = "./data/les_trois_mousquetaire.txt"

text = []
with open(txt_filename, "r", encoding='utf-8') as input_f:
    text = input_f.read()

# split text by paragraph

paragraphs = re.split("(?<!\")\n\n(?<!\")", text)#\n is the name for skipping to the next line
print("The text is composed of {} paragraphs!\n".format(len(paragraphs)))

# extract all individual sentences from a text
def sentences_in(text):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    sentences = sent_detector.tokenize(text.strip())
    return sentences

# extract all the dialogue within quotations in a text
def dialogue_in(text):
    normalQuotes = re.findall('"([^"]*)"', text)
    directionalQuotes = re.findall('“([^”]*)”', text)
    return normalQuotes + directionalQuotes

# extract all words beginning with a capital letter in a text
def proper_names_in(text):
    return re.findall("[A-Z][a-z]+", text)

# test with a random paragraph
n = 75

print(paragraphs[n])
dialogues = dialogue_in(paragraphs[n])
sentences = sentences_in(paragraphs[n])
names = proper_names_in(paragraphs[n])
print("\nParagraph {} is composed of {} sentences, {} dialogues and {} proper names".format(
    n, len(sentences), len(dialogues), len(names)))
print("The first sentence is: \n", sentences[0])
if(len(dialogues)!=0):
    print("The first dialogue is: \n", dialogues[0])
print("The first 'name' is: \n", names[0])

- As we can see finding the proper names of characters is no easy task, so we decided to rely on a curated external list of characters to look for matches and build our graph.

In [ ]:
# let's list all the proper names in the book. This might take a while...
allNames = set(proper_names_in(text))
print(allNames)
# need to make a decision here

In [ ]:
#open character list
#txt_filename = "./data/CharacterList.txt"
txt_filename = "./data/name_list.txt"

characterList = []
with open(txt_filename, "r") as input_f:
    characterList = input_f.read()

#parsing characterlist and counting them
mainCharacters = re.split("\n", characterList)
totalCharacters = len(mainCharacters)
print(mainCharacters)

# making dictionary of associations {graph node : character}
nodeCharacter = dict()
for i in range(len(mainCharacters)):
    nodeCharacter[i] = mainCharacters[i]
    
print("\n Node associations: ", nodeCharacter)

In [ ]:
#cleans a list of characters !!!TODO eliminate duplicates
def clean_names(characterList, mainCharacters):
    for character in list(characterList):
        if character not in mainCharacters:
            characterList.remove(character)
            
    
#iterate over cleant list and updates matrix (lamest optimization ever)
def update_matrix(characterList, mainCharacters, matrix):
    if len(characterList) < 2:
        return
    
    for i in range(len(characterList)):
        currentCharacterIndex = mainCharacters.index(characterList[i])
        for j in range(len(characterList) - i - 1):
            secondCharacterIndex = mainCharacters.index(characterList[j + i + 1])
            matrix[currentCharacterIndex][secondCharacterIndex] += 1;
            matrix[secondCharacterIndex][currentCharacterIndex] += 1;
            
            

#matrix totalCharacters * totalCharacters initialized with all 0s
adjacencyMatrix = [[0 for _ in range(totalCharacters)] for _ in range(totalCharacters)]

#iterate over paragraphs and get proper names
for i in range(len(paragraphs)):
    paragraphCharacters = proper_names_in(paragraphs[i])
    paragraphCharacters = list( dict.fromkeys(paragraphCharacters))
    clean_names(paragraphCharacters, mainCharacters)
    update_matrix(paragraphCharacters, mainCharacters, adjacencyMatrix)
    
print(adjacencyMatrix)


- Making the graph

In [ ]:
G = nx.Graph()

# add one node per character
for i in range(len(mainCharacters)):
    G.add_node(i, size = 1)
    
    
# add node labels
nx.set_node_attributes(G, nodeCharacter, name='Names')

# add the edges from the adjacency matrix (avoid duplicates)

for i in range(len(adjacencyMatrix)-1):
    for j in range(i+1, len(adjacencyMatrix[0])):
        if adjacencyMatrix[i][j] != 0 :
            G.add_edge(i,j, weight = adjacencyMatrix[i][j])
    
    
    
print(nx.info(G))
#print(G.nodes[0][""])

def print_graph(G):
    g_draw = G

    fig = plt.figure(figsize=(10,10))
    pos = nx.circular_layout(g_draw)
    nx.draw(g_draw, pos, node_color='lightblue', edge_color='lightblue', node_size=50, labels = nx.get_node_attributes(G, 'Names'), with_labels = True, width=0.5)
    
    
    
print_graph(G)
plt.savefig('./data/socialNetwork.png',dpi=300)
plt.show()

<font size=4> Preliminary observations about the graph: </font>

- Graph shape





- Degree assortativity: disassortative network

In [ ]:
def  find_assortativity(graph):
    return nx.degree_assortativity_coefficient(graph)

assortativity = find_assortativity(G)
print(assortativity)

- Community detection with Louvain algorithm and Spectral clustering

In [ ]:
#Louvain algorithm

communities1 = nx_comm.louvain_communities(G)
print("Louvain algo detected: ", communities1)

# Spectral clustering

#prepare the adjacency matrix
npMat = np.array([np.array(row) for row in adjacencyMatrix])
"""
for i in range(npMat.shape[0]):
    for j in range(npMat.shape[1]):
        if npMat[i][j] != 0:
            npMat[i][j] = 1
"""
# Compute Laplacian matrix and set k to the number of eigenvalues "close" to zero -> optmal k
Laplacian = npMat.copy()

# edges are only counted with weight 1 for the Laplacian

for i in range(Laplacian.shape[0]):
    for j in range(Laplacian.shape[1]):
        if Laplacian[i][j] != 0:
            Laplacian[i][j] = -1

for i in range(len(Laplacian)):
    Laplacian[i][i] = nx.degree(G, i)

# there should be at least one ZERO eigenvalue, then k-1 eignevalues CLOSE to zero
w,v = LA.eig(Laplacian)
print("Laplacian eigenvalues : ", w)

#for the three musketeers, it looks like there is 1 eigenvalue close or equal to zero
k=1

clustering = sklearn.cluster.SpectralClustering(n_clusters=k, assign_labels='discretize', random_state=0, affinity='precomputed_nearest_neighbors').fit(npMat)
communities2 = clustering.labels_
print("SKL Spectral Clustering algo detected communities with labels: ", communities2)

# plot the communities
def plot_louvain_comm(G, community_):
    color_map = []
    # color up to 10 communities
    color_list = ['blue', 'green', 'red', 'yellow', 'purple', 'orange', 'cyan', 'brown', 'black', 'white']
    for node in G:
        for i in range(len(community_)):
            if node in community_[i]:
                color_map.append(color_list[i])
    
    g_draw = G          
    fig = plt.figure(figsize=(10,10))
    pos = nx.circular_layout(g_draw)
    nx.draw(g_draw, pos, node_color=color_map, edge_color='lightblue', node_size=50, with_labels=True, width=0.5)
    plt.savefig('./data/louvainCommunities.png',dpi=300)
    plt.show()
    
def plot_spectral_comm(G, community_):
    color_map = []
    # color up to 10 communities
    color_list = ['blue', 'green', 'red', 'yellow', 'purple', 'orange', 'cyan', 'brown', 'black', 'white']
    for i in range(len(G.nodes)):
        color_map.append(color_list[community_[i]])
    
    g_draw = G          
    fig = plt.figure(figsize=(10,10))
    pos = nx.circular_layout(g_draw)
    nx.draw(g_draw, pos, node_color=color_map, edge_color='lightblue', node_size=50, with_labels=True, width=0.5)
    plt.savefig('./data/spectralCommunities.png',dpi=300)
    plt.show()
    
print("Plot of louvain communities:")
plot_louvain_comm(G, communities1)


print("Plot of spectral communities:")
plot_spectral_comm(G, communities2)

- Comparison of the two methods



<font size=4> Influence maximization problem </font>

IC model from: https://github.com/AdnanRasad/Influence-Maximization-Analysis/blob/master/Kempe-Independent-Cascades-Model/IC-Graph:Networkx-Python

tuple to list: https://stackoverflow.com/questions/12142133/how-to-get-first-element-in-a-list-of-tuples

In [ ]:
# k = int(G.number_of_nodes() * 0.05)

k = 3

print(k)

# simulates Num_of_Simulations loops of Independent Cascade
def IC(Networkx_Graph,Seed_Set,Probability,Num_of_Simulations):
    spread = []
    Y = []
    
    for i in range(Num_of_Simulations):
        Y.append([])
        Y[i].append(len(Seed_Set) / len(G.nodes))
        new_active, Ans = Seed_Set[:], Seed_Set[:]
        while new_active:
            #Getting neighbour nodes of newly activate node
            targets = Neighbour_finder(Networkx_Graph,Probability,new_active)
    
            #Calculating if any nodes of those neighbours can be activated, if yes add them to new_ones.
            np.random.seed(i)
            success = np.random.uniform(0,1,len(targets)) < Probability
            new_ones = list(np.extract(success, sorted(targets)))
            
            #Checking which ones in new_ones are not in our Ans...only adding them to our Ans so that no duplicate in Ans.
            new_active = list(set(new_ones) - set(Ans))
            Ans += new_active
            infOverTot = len(Ans) / len(G.nodes)
            Y[i].append(infOverTot)
        
        spread.append(len(Ans))
    
    maxRowLenght = Max_Row_Len(Y)
    Even_Rows(Y, maxRowLenght)
    return(np.mean(spread), Y, np.std(spread))
      

#finds all the neighbors available for a set of nodes
def Neighbour_finder(g,p,new_active):
    
    targets = []
    for node in new_active:
        targets += g.neighbors(node)

    return(targets)


#returns the max len among the rows
def Max_Row_Len(listOfLists):
    lenghts = []
    
    for i in range(len(listOfLists)):
        lenghts.append(len(listOfLists[i]))
        
    return max(lenghts)


#Even the rows out appending the last element to get them maxLenght long
def Even_Rows(listOfLists, maxLenght):    
    for i in range(len(listOfLists)):
        elementsToAdd = maxLenght - len(listOfLists[i])
        currentLen = len(listOfLists[i])
        lastIndex = max(currentLen - 1, 0)
        lastElement = listOfLists[i][lastIndex]
        
        for j in range(elementsToAdd):
            listOfLists[i].append(lastElement)
        

#TODO find an adequate influence function for a set of nodes A
#Run IC a 'large' number of times to find realizations X_i (By 'large' I mean like 10 let's not make this take forever)
#Compute probability for each realization X_i, still unclear on how to do that because the notes are vague af

def Greedy_Influence_Maximization(G, k, Probability, Num_of_Simulations):
    A0 = []
    V = G.nodes
    while(len(A0) < k):
        availableNodes = V - A0
        max = 0
        for node in availableNodes:
            value = IC(G, A0 + [node], Probability, Num_of_Simulations)[0]
            if (value > max):
                max = value
                v = node
        A0 = A0 + [v]
    return A0

maxInfluenceSet = Greedy_Influence_Maximization(G, k, Probability=0.5, Num_of_Simulations=4)

print(maxInfluenceSet)

#Compare runs if IC starting from MI set to other runs starting from k max degree nodes

MI_spread = IC(G, maxInfluenceSet, Probability=0.5, Num_of_Simulations=5044)

#Extract the set of 3 nodes of maximum degree
maxDegSet = sorted(G.degree, key=lambda tup: tup[1], reverse=True)[:len(maxInfluenceSet)]
maxDegSet = [i[0] for i in maxDegSet]

MaxDegSpread = IC(G, maxDegSet, Probability=0.5, Num_of_Simulations=4)

#print(MaxDegSpread)
#print(MI_spread)

def plot_maxinf_set(G, set_):
    color_map = []
    # color up to 10 communities
    color_list = ['lightblue', 'red']
    for node in G:
        if node in set_:
            color_map.append(color_list[1])
        else:
            color_map.append(color_list[0])
    
    g_draw = G          
    fig = plt.figure(figsize=(10,10))
    pos = nx.circular_layout(g_draw)
    nx.draw(g_draw, pos, node_color=color_map, edge_color='lightblue', node_size=50, with_labels=True, width=0.5)
    

    
plot_maxinf_set(G, maxInfluenceSet)
plt.savefig('./data/maxInfluenceSocial.png',dpi=300)
plt.show()

In [ ]:
print(MaxDegSpread[1])

In [ ]:
import matplotlib.pyplot as plt


#makes the all the rows at the same lenght (evens out epochs for all the tests)
maxRowLenght = max(len(MaxDegSpread[1][0]), len(MI_spread[1][0]))
Even_Rows(MaxDegSpread[1], maxRowLenght)
Even_Rows(MI_spread[1], maxRowLenght)

#converts Y to numpy array
yMiSpread = np.array(MI_spread[1])
yMaxDegSpread = np.array(MaxDegSpread[1])

yMiSpread = np.mean(yMiSpread, axis=0)
yMaxDegSpread = np.mean(yMaxDegSpread, axis=0)


#plot graph

x = range(0, maxRowLenght)

plt.plot(x, yMaxDegSpread, label = "MaxDegSpread")
plt.plot(x, yMiSpread, label = "MiSpread")
plt.legend()
plt.savefig('./data/spreadGraph.png',dpi=300)
plt.show()





- Barabasi-Albert comparison

In [ ]:
#Generate graph with same number of nodes and average degree

H = nx.barabasi_albert_graph(len(G.nodes), int(np.mean(G.degree)))

print_graph(H)
plt.savefig('./data/barb-alb-graph.png',dpi=300)
plt.show()

Compute the Greedy Influence Maximization algorithm on the new network

In [ ]:
BarbMaxInfluenceSet = Greedy_Influence_Maximization(H, k, Probability=0.5, Num_of_Simulations=4)
print(BarbMaxInfluenceSet)


plot_maxinf_set(H, BarbMaxInfluenceSet)
plt.savefig('./data/barbMaxInfluenceSocial.png',dpi=300)
plt.show()